In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.neighbors import NearestNeighbors
from gensim.models import KeyedVectors

## Part B Data

Will need HCPCS codes and their descriptions.

In [2]:
partb = pd.read_csv(os.environ['CMS_PARTB_PATH'], usecols=['hcpcs_code', 'hcpcs_description'])
unique_hcpcs = partb['hcpcs_code'].unique()

In [3]:
partb = partb.groupby('hcpcs_code').agg({ 'hcpcs_description': 'first' }).reset_index()

In [4]:
hcpcs_definitions = {}

for _, row in partb.iterrows():
  hcpcs_definitions[row.hcpcs_code] = row.hcpcs_description

In [ ]:
'00140'
'11755'
'20926': 'Tissue graft'
  '25320': 'Repair of wrist joint'
38900 Lymph node imaging during surgery
83904 Genetic test
J3120 Injection, testosterone enanthate, up to 100 mg
23465 Repair of shoulder joint
70557 MRI scan of brain
71030 X-ray of chest, minimum of 4 views
97003 Occupational therapy evaluation
99281 Emergency department visit, self limited or minor problem
84132 Blood potassium level


## Skipgram 300

In [9]:
%ls ../../HCPCS-Embeddings-For-Fraud-Detection/data

cbow-e150-w10-i200.kv      skipgram-e150-w10-i100.kv
cbow-e150-w5-i200.kv       skipgram-e150-w5-i100.kv
cbow-e300-w10-i200.kv      skipgram-e300-w10-i100.kv
cbow-e300-w5-i200.kv       skipgram-e300-w5-i100.kv
cbow-e75-w10-i200.kv       skipgram-e75-w10-i100.kv
cbow-e75-w5-i200.kv        skipgram-e75-w5-i100.kv
choi-mcemc-e300.kv         uniform-e75.pickle
choi-mcemc-e300.pickle


In [20]:
hcpcs_eval = ['83904', '20926', '23465', 'J3120', '70557', '84132', '71030']

def get_neighbors(embeddings):
  for hcpcs in hcpcs_eval:
    print(hcpcs, hcpcs_definitions[hcpcs])
    top3 = np.argsort(embeddings.cosine_similarities(embeddings[hcpcs], embeddings.vectors))[-4:-1]
    top3hcpcs = [embeddings.index2word[x] for x in top3]
    top3descriptions = [hcpcs_definitions[x] for x in top3hcpcs]
    for result in zip(top3hcpcs, top3descriptions):
      print(result)
    print()

In [23]:
embeddings = KeyedVectors.load('../../HCPCS-Embeddings-For-Fraud-Detection/data/cbow-e300-w5-i200.kv')

get_neighbors(embeddings)

83904 Genetic test
('83897', 'Genetic test')
('88386', 'Evaluation of genetic test')
('83894', 'Genetic test')

20926 Tissue graft
('29879', 'Repair of knee joint using an endoscope')
('20900', 'Small bone graft from any donor area')
('29876', 'Removal of joint lining from two or more knee joint compartments using an endoscope')

23465 Repair of shoulder joint
('23929', 'Shoulder procedure')
('23450', 'Reattachment of shoulder joint capsule')
('23474', 'Repair of shoulder')

J3120 Injection, testosterone enanthate, up to 100 mg
('A9516', 'Iodine i-123 sodium iodide, diagnostic, per 100 microcuries, up to 999 microcuries')
('J3411', 'Injection, thiamine hcl, 100 mg')
('J1000', 'Injection, depo-estradiol cypionate, up to 5 mg')

70557 MRI scan of brain
('61582', 'Removal of facial bone to approach brain lesion')
('22207', 'Incision of spine to correct deformity at lower spinal column')
('61864', 'Implantation of neurostimulator electrodes in brain')

84132 Blood potassium level
('82465',

In [22]:
embeddings = KeyedVectors.load('../../HCPCS-Embeddings-For-Fraud-Detection/data/skipgram-e75-w10-i100.kv')

get_neighbors(embeddings)

83904 Genetic test
('81287', 'MGMT (O-6-methylguanine-DNA methyltransferase) gene analysis')
('88283', 'Chromosome analysis for genetic defects')
('83894', 'Genetic test')

20926 Tissue graft
('15770', 'Creation of skin, fat and muscle graft')
('20902', 'Bone graft harvest')
('20900', 'Small bone graft from any donor area')

23465 Repair of shoulder joint
('23466', 'Repair of shoulder joint')
('24301', 'Relocation of muscle or tendon of upper arm or elbow')
('23397', 'Relocation of muscles of shoulder or upper arm')

J3120 Injection, testosterone enanthate, up to 100 mg
('J1070', 'Injection, testosterone cypionate, up to 100 mg')
('J1080', 'Injection, testosterone cypionate, 1 cc, 200 mg')
('J3130', 'Injection, testosterone enanthate, up to 200 mg')

70557 MRI scan of brain
('61751', 'Stereotactic biopsy, aspiration, or excision of brain lesion using CT and/or MRI guidance')
('70559', 'MRI scan of brain before and after contrast')
('61880', 'Revision or removal of brain neurostimulator